In [2]:
import numpy as np
import pandas as pd

import seaborn as sns

%matplotlib widget
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [10,8]
pd.options.display.max_columns = None
sns.set(style="ticks", color_codes=True)


In [3]:
df = pd.read_pickle('./Data/process_data_df10.pkl')

`pa_coating` is a dummy variable and doesn't belong to the original collected data. For further analysis it can be dropped.

### Removing unnecessary columns

In [4]:
colsToRemove = ['pa_coating']

df1 = df.copy()
df1.drop(labels=colsToRemove,axis='columns',level=1,inplace=True)

Let's check whether the variables where succesfully removed:

In [5]:
# Check if any elements of colsToRemove is in df
print('Are any of the `colsToRemove` in `df`?')
print('  ',any([i for i in colsToRemove if i in df.columns.get_level_values(level=1)]))
print('\n')

# Check if any elements of colsToRemove is in df1
print('Are any of the `colsToRemove` in `df1`?')
print('  ',any([i for i in colsToRemove if i in df1.columns.get_level_values(level=1)]))

Are any of the `colsToRemove` in `df`?
   True


Are any of the `colsToRemove` in `df1`?
   False


### Exploring `p_product`, `p_product_type` and `p_product_group` 

Nice! Now we could keep removing unnecessary columns, but let's focus in the feature we want to keep/analyse.

My 1st question is: 
> Do the features `qc_salzrckhalt` and `qc_durchfluss` vary significantly among different `p_product`, `p_product_type` and `p_product_group`? 

In [6]:
# df1.head()
# df1.loc[:,(slice(None),['ps','nr','p_product','p_product_group','p_product_type','qc_salzrckhalt','qc_durchfluss'])].head()

In [7]:
df2 = df1.copy()
df2 = df2.droplevel(level= 0, axis='columns')

XY  = df2[['nr','p_product_group','p_product_type','p_product','qc_salzrckhalt','qc_durchfluss']]
# Y  = df2[['qc_salzrckhalt','qc_durchfluss']]
# X  = df2[['nr','p_product','p_product_group','p_product_type']]

In [8]:
groupCols = ['p_product_group','p_product_type','p_product']
goalCols  = ['qc_salzrckhalt','qc_durchfluss']

XY_mean = XY.groupby(groupCols)[goalCols].mean()
XY_var  = XY.groupby(groupCols)[goalCols].var()
# pd.concat({'mean':XY_mean,'var':XY_var},axis=1)

pd.merge(XY_mean,XY_var,
         sort=True,
         suffixes=('_mean','_var'),
         left_index=True,
         right_index=True)

# pd.options.display.max_rows = 10000
pd.reset_option("display.max_rows")
XY_mean.join(XY_var,
      lsuffix='_mean',
      rsuffix='_var')

qc_salzrckhalt_mean  \
p_product_group     p_product_type p_product                            
Brackwasserelemente 5200bdfc01a1   B400 HP                  98.989131   
                                   B440 HP                  99.163510   
                    6989995295da   B085 LE 4040             99.144588   
                                   B400 LE                  98.935167   
                                   B400 LE ASD              99.055540   
                                   B440 LE                  98.782107   
                    6d2830b1e76d   B085 HF 4040             99.113559   
                                   B400 HF                  99.012014   
                                   B440 HF                  98.694982   
                    c77cb1692e0e   B085 ULP 4040            99.271320   
                                   B400 ULP ASD             98.050863   
                                   B440 ULP                 98.218921   
                    f187928fdb22   B370 FR                  98.812073   
                                   B400 FR ASD              99.013156   
                    f8ba29e9058b   B085 FR 4040             98.701824   
                                   B400 FR                  98.924073   
Seewasserelemente   320144db7b4b   S085 HF 4040             98.616838   
                                   S400 HF                  96.226096   
                                   S440 HF                  99.343883   
                    cd7216b44ce3   S085 HR 4040             98.377514   
                                   S400 HR                  96.653459   
                                   S440 HR                  98.344120   

                                                  qc_durchfluss_mean  \
p_product_group     p_product_type p_product                           
Brackwasserelemente 5200bdfc01a1   B400 HP                 47.087445   
                                   B440 HP                 54.066717   
                    6989995295da   B085 LE 4040             7.833652   
                                   B400 LE                 35.869479   
                                   B400 LE ASD             38.267354   
                                   B440 LE                 40.682883   
                    6d2830b1e76d   B085 HF 4040            10.152490   
                                   B400 HF                 45.510025   
                                   B440 HF                 47.145020   
                    c77cb1692e0e   B085 ULP 4040            8.298521   
                                   B400 ULP ASD            41.327267   
                                   B440 ULP                45.644223   
                    f187928fdb22   B370 FR                 40.580885   
                                   B400 FR ASD             46.633970   
                    f8ba29e9058b   B085 FR 4040            10.073621   
                                   B400 FR                 44.725683   
Seewasserelemente   320144db7b4b   S085 HF 4040             7.088446   
                                   S400 HF                 33.766855   
                                   S440 HF                 39.621275   
                    cd7216b44ce3   S085 HR 4040             6.265555   
                                   S400 HR                 27.447772   
                                   S440 HR                 25.526313   

                                                  qc_salzrckhalt_var  \
p_product_group     p_product_type p_product                           
Brackwasserelemente 5200bdfc01a1   B400 HP                 71.476345   
                                   B440 HP                 51.370364   
                    6989995295da   B085 LE 4040            44.251208   
                                   B400 LE                 66.230018   
                                   B400 LE ASD             53.562097   
                                   B440 LE                 86.79

In [9]:
groupCols = ['p_product_group','p_product_type','p_product']
goalCols  = ['qc_salzrckhalt','qc_durchfluss']

(XY
 .query("qc_salzrckhalt>0")
 .quary("qc_durchfluss>0")
 .groupby(groupCols)[goalCols]
 .describe()
)

qc_salzrckhalt             \
                                                          count       mean   
p_product_group     p_product_type p_product                                 
Brackwasserelemente 5200bdfc01a1   B400 HP               9876.0  99.710801   
                                   B440 HP               1918.0  99.680525   
                    6989995295da   B085 LE 4040          6041.0  99.587711   
                                   B400 LE               2091.0  99.597574   
                                   B400 LE ASD           5330.0  99.594492   
                                   B440 LE               6446.0  99.655607   
                    6d2830b1e76d   B085 HF 4040          4849.0  99.747199   
                                   B400 HF              21935.0  99.725208   
                                   B440 HF              24132.0  99.709252   
                    c77cb1692e0e   B085 ULP 4040         1891.0  99.481307   
                                   B400 ULP ASD          8073.0  99.447599   
                                   B440 ULP              3871.0  99.487571   
                    f187928fdb22   B370 FR               5977.0  99.754400   
                                   B400 FR ASD           1035.0  99.778475   
                    f8ba29e9058b   B085 FR 4040          1410.0  99.751843   
                                   B400 FR               7154.0  99.698430   
Seewasserelemente   320144db7b4b   S085 HF 4040           202.0  99.593242   
                                   S400 HF               3025.0  99.661606   
                                   S440 HF                450.0  99.564647   
                    cd7216b44ce3   S085 HR 4040           237.0  99.622799   
                                   S400 HR               1468.0  99.550429   
                                   S440 HR                148.0  99.673094   

                                                                       \
                                                       std        min   
p_product_group     p_product_type p_product                            
Brackwasserelemente 5200bdfc01a1   B400 HP        0.178502  91.557525   
                                   B440 HP        0.274059  89.335492   
                    6989995295da   B085 LE 4040   0.558698  65.393205   
                                   B400 LE        0.817313  62.738794   
                                   B400 LE ASD    0.408559  82.375931   
                                   B440 LE        0.705622  61.189250   
                    6d2830b1e76d   B085 HF 4040   0.851215  63.662601   
                                   B400 HF        0.606740  57.286870   
                                   B440 HF        0.423277  69.786252   
                    c77cb1692e0e   B085 ULP 4040  0.127754  98.782418   
                                   B400 ULP ASD   2.260389   7.349495   
                                   B440 ULP       2.190241  24.139359   
                    f187928fdb22   B370 FR        0.531778  70.834117   
                                   B400 FR ASD    0.111351  96.996042   
                    f8ba29e9058b   B085 FR 4040   0.569136  84.925017   
                                   B400 FR        0.943967  60.317421   
Seewasserelemente   320144db7b4b   S085 HF 4040   0.322570  97.279632   
                                   S400 HF        0.121766  98.046078   
                                   S440 HF        0.250485  98.214860   
                    cd7216b44ce3   S085 HR 4040   0.192800  98.306194   
                                   S400 HR        0.283806  97.296347   
                                   S440 HR        0.126316  99.066390   

                                                                        \
                                                        25%        50%   
p_product_group     p_product_type p_product                             
Brackwasserelemente 5200bdfc01a1   B400 HP        

In [9]:
groupCols = ['p_product_group','p_product_type','p_product']
goalCols  = ['qc_salzrckhalt','qc_durchfluss']

XY_mean = XY.groupby(groupCols)[goalCols].mean()
XY_std  = XY.groupby(groupCols)[goalCols].std()

smry= XY_mean.join(XY_std,
      lsuffix='_mean',
      rsuffix='_std')

dir(smry.index)

smry.index.get_level_values(level='p_product_group')

#smry['p_product_group']

%matplotlib widget

fig1 = plt.figure(figsize=[6,4])
plt.scatter(smry.index.get_level_values(level='p_product_group'),smry['qc_salzrckhalt_mean'])
plt.show()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
#sns.catplot(y='p_product_type',x='qc_salzrckhalt',hue='p_product_group',data=XY)
sns.catplot(y='p_product',x='qc_salzrckhalt',hue='p_product_type',data=XY[XY['p_product_group'] == 'Brackwasserelemente'])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
(XY
 .query("qc_salzrckhalt>0") # filter out zero values, since they appear to not make sense, i.e. 
 .dropna()                   # drop missing values
 .pipe((sns.catplot,'data'),
       y='p_product',x='qc_salzrckhalt',col='p_product_type',col_wrap = 4,hue='p_product_type',kind='strip'))

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
(XY
 .query("qc_salzrckhalt>0")
 .dropna()
 .pipe((sns.catplot,'data'),
       x='qc_durchfluss',y='p_product',hue='p_product_group'))

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [34]:
fig, ax = plt.subplots()
ax = (XY
 .query("qc_salzrckhalt>0")
 .dropna()
 .pipe((sns.scatterplot,'data'),
       x='qc_salzrckhalt',y='nr',hue='p_product_group',alpha=0.3))

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Categorical variables:
**Is there a difference in the distribution of `qc_salzrckhalt` or `qc_durchfluss`**?
 
 product type, group,etc 